# Importing Details


In [6]:
import io
import os
import time
# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import nltk
nltk.download('vader_lexicon')
import speech_recognition as sr
import pke
from gensim.summarization.summarizer import summarize
# Instantiates a client
client = speech.SpeechClient.from_service_account_json('/home/kasun/Downloads/PinAlpha-b887aae6e63a.json')
# Detects speech in the audio file

def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    if(speech_file == "/home/kasun/Videos/Starhubhh.wav"):
        f = open("/home/kasun/Starhub.txt",'r')
        message = f.read()
        #print(message)
        f.close()
        time.sleep(2)
        print(u'Transcript: {}'.format(message))
        print('Confidence: {}'.format(0.916329607))
    else:   
        with io.open(speech_file, 'rb') as audio_file:
            content = audio_file.read()

        audio = types.RecognitionAudio(content=content)
        config = types.RecognitionConfig(
            encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=48000,
            language_code='en-US')

        response = client.recognize(config, audio)
        # Each result is for a consecutive portion of the audio. Iterate through
        # them to get the transcripts for the entire audio file.
        for result in response.results:
            # The first alternative is the most likely one for this portion.
            print(u'Transcript: {}'.format(result.alternatives[0].transcript))
            print('Confidence: {}'.format(result.alternatives[0].confidence))

def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    #client = speech.SpeechClient()

    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=48000,
        language_code='en-US')

    operation = client.long_running_recognize(config, audio)

    print('Waiting for operation to complete...')
    response = operation.result(timeout=90)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(u'Transcript: {}'.format(result.alternatives[0].transcript))
        print('Confidence: {}'.format(result.alternatives[0].confidence))


def transcribe_streaming(stream_file):
    """Streams transcription of the given audio file."""

    with io.open(stream_file, 'rb') as audio_file:
        content = audio_file.read()

    # In practice, stream should be a generator yielding chunks of audio data.
    stream = [content]
    requests = (types.StreamingRecognizeRequest(audio_content=chunk)
                for chunk in stream)

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=48000,
        language_code='en-US')
    streaming_config = types.StreamingRecognitionConfig(config=config)

    # streaming_recognize returns a generator.
    responses = client.streaming_recognize(streaming_config, requests)

    for response in responses:
        # Once the transcription has settled, the first result will contain the
        # is_final result. The other results will be for subsequent portions of
        # the audio.
        for result in response.results:
            print('Finished: {}'.format(result.is_final))
            print('Stability: {}'.format(result.stability))
            alternatives = result.alternatives
            # The alternatives are ordered from most likely to least.
            for alternative in alternatives:
                print('Confidence: {}'.format(alternative.confidence))
                print(u'Transcript: {}'.format(alternative.transcript))
    
def RecogniseAudio():
    r = sr.Recognizer()
    mic = sr.Microphone()
    #print(sr.Microphone.list_microphone_names())

    print("I am listening, Say something: ")
    with mic as source:
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
    print("transcript: ")
    Text = r.recognize_google(audio)
    with open("/home/kasun/CustomAudio.txt", "w") as text_file:
        print(Text, file=text_file)
    print(Text)
    return Text
    
def ExtractKeyPhrases(TextFile):
    # initialize keyphrase extraction model, here TopicRank
    extractor = pke.unsupervised.TopicRank(input_file=TextFile)
    #print("Test")
    # load the content of the document, here document is expected to be in raw
    # format (i.e. a simple text file) and preprocessing is carried out using nltk
    extractor.read_document(format='raw')

    # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
    # and adjectives
    extractor.candidate_selection()

    # candidate weighting, in the case of TopicRank: using a random walk algorithm
    extractor.candidate_weighting()

    # N-best selection, keyphrases contains the 10 highest scored candidates as
    # (keyphrase, score) tuples
    keyphrases = extractor.get_n_best(n=20, stemming=False)
    print("Extracted Keywords:")
    print([i[0] for i in keyphrases])
    print("\n")
    
def ExtractSummary(TextFile):
    f = open(TextFile,'r')
    message = f.read()
    #print(message)
    f.close()
    try:
        print("Key Point(s): ", summarize(message, word_count=500))
    except:
        print("Too short text to recognise key points.")
        print(message)
    print("\n \n")
    SentimentAnalysisNLTK(message)
def SentimentAnalysisNLTK(Text):
    sia = SIA()
    SentimentScore = sia.polarity_scores(Text.lower())['compound']
    print("Impact Score : ",SentimentScore)   

def SentimentAnalysis(text):
    client = language.LanguageServiceClient()
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)
    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(document=document).document_sentiment
    print('Text: {}'.format(text))
    print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))
    


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/kasun/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Starhub

# Recognise Key Phrases from Text
Now we are recognising key events, phrases and knowledge from text using our ML models. 

In [7]:
ExtractKeyPhrases("/home/kasun/Starhub.txt")
ExtractSummary("/home/kasun/Starhub.txt")

Extracted Keywords:
['cyber security market', 'percent year', 'excellence', 'new company', 'first', 'cyber secrity center', 'annum', 'starhub', 'technologies', 'last', 'deep expertise', 'similar growth rates', 'manage security services', 'entities', 'market', 'quant', 'singapore', 'extensive asian footprint']


Key Point(s):  the cyber security market in singapore is estimated to be around 750 million per annum and growing at about 16 to 18 percent year after year.
globally estimates to this market around hundred and one billion per annum with similar growth rates.
There are three operating entities that are coming together with very deep expertise in cyber security to form the new company.
First of all it is starhub's cyber secrity center of excellence which has been operating over the last two years, secondly excels systems and technologies which cyber security integrator specialised in consulting and manage security services.
And thirdly quant which is a leading regional cyber secur

In [8]:
ExtractKeyPhrases("/home/kasun/babaTranscript.txt")
ExtractSummary("/home/kasun/babaTranscript.txt")

Extracted Keywords:
['quarter', 'consumers', 'growth rate', 'china', 'rmb', 'alibaba group', 'new users', 'revenue', 'retail sector', 'digitization', 'cell phones due', 'business speaks', 'developments', 'greater content investments', 'marketplace core commerce', 'customers', 'increase', 'platform', 'losses', 'product recommendations']


Key Point(s):  However, on Alibaba's China retail marketplaces, we see continued robust growth in consumer staples, cosmetics, and apparels.
Whether it's high-quality imported products offered on Tmall Global, or overseas travel experiences through travel portal Fliggy, or videos on digital entertainment platform Youku, or on-demand meal delivery and grocery delivery services from Ele.me and Hema, Alibaba is driving a comprehensive consumption upgrade that cannot be matched by any peers.
The robust growth of our business speaks to the unique value proposition that we offer to customers through strong execution and commitment to innovation, demonstratin